<a href="https://colab.research.google.com/github/sheltauro/makemore/blob/main/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-04-13 08:35:55--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-04-13 08:35:55 (19.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [4]:
print("length of the dataset in characters: ", len(text))

length of the dataset in characters:  1115394


In [6]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [12]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [28]:
itos = {i:ch for i,ch in enumerate(chars)}
stoi = {ch:i for i,ch in enumerate(chars)}

encode = lambda s: [stoi[ch] for ch in s] # encoder: take a string, output a list of integers.
decode = lambda l: ''.join([itos[num] for num in l])

print(encode("hello"))
print(decode(encode("hello")))

[46, 43, 50, 50, 53]
hello


In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [40]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [69]:
torch.manual_seed(1337)
block_size = 8 # what is the maximum context length for predictions.
batch_size = 4 # how many independent character sequences should we process in parallel.

def get_batch(split):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data) - block_size, (batch_size, ))
  x = torch.stack([data[i:i+block_size] for i in ix], dim=0)
  y = torch.stack([data[i+1:i+block_size+1] for i in ix], dim=0)
  return x, y

# offset = 5
# new_list = train_data[offset:block_size + 1 + offset]

# x, y = new_list[:block_size], new_list[1:block_size+1]

xb, yb = get_batch('train')
print(xb.shape, yb.shape)
print(f"inputs: {xb}")

for b in range(batch_size): # batch dimension
  for t in range(block_size): # time(context) dimension
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")


torch.Size([4, 8]) torch.Size([4, 8])
inputs: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1, 58, 46] the target: 39
when input is [44, 53, 56, 1, 58, 46, 39] the target: 58
when input is [44, 53, 56, 1, 58, 46, 39, 58] the target: 1
when input is [52] the target: 58


In [63]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    # creating an embedding lookup table.
    # each token directly reads off the logits for the next token from a lookup table.
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # (B, T) -> (B, T, C) where C = vocab_size
  
  def forward(self, idx, targets=None):

    # idx and targets are both (B, T) tensor of integers.
    logits = self.token_embedding_table(idx)
    if targets != None:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
      return logits , loss
    else:
      return logits

  def generate(self, idx, max_new_tokens):
    # idx is (B,T) array of indices in the current context.
    for _ in range(max_new_tokens):
      logits = self(idx)
      # focus only on the last time step.
      logits = logits[:, -1, :]
      # apply softmax to get prediction probabilities.
      probs = F.softmax(logits, dim=1) # B, C
      # sample from the probability distribution.
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, idx_next), dim = 1) # (B, T+1)
    return idx
      


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [72]:
idx = torch.zeros((1, 1), dtype=torch.long)
out = m.generate(idx, 100)
print(out.shape)
print(decode(out[0].tolist()))

torch.Size([1, 101])

.O
jFabjgH.BLpYjWKpvo.e?pxKbOjtLI!ehsstjRyQ?niv,.bL&GXGUv?YV:zP:p$
gwWyCVKHZJuk-wW j-wWPQBc.ykxz?wsB


In [74]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [81]:
# start the training woohoo!!

batch_size = 32
for steps in range(1000):
  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.46651029586792


In [83]:
print(decode(m.generate(idx, 1000)[0].tolist()))


KIO: ere ofarced ay fepo s bouf g t!
S:
Sited mCI'Thun h He siseKAUDY:
AruD'dwns, m andWhad SThir!
Thot
Ty, MIUNod, hade YoZhen meapsivevy Wis heire oaplkeefell'Tho UDuen und s lvVO:

J tcke wasaros KI s tr Incors COMp,
Whbesth mar ENMain anghed e outusthIVall pJhourlly

Whep manks aiAY cin
BLIf:
w:
OR:
Minthay w g Coutalonisece nt

LUEDWhick!
Lf pe oku ANULIVI
QbthiKA athol wat Be n you.
RDurunomenogZvof d pallis?-isof y lllt.
Grkliso Chst t; -Gounoumy andicovesit asthen'd ay hasat RO:
YGLONone m-ews m? oghy Myoth isoubein is k!
Acot:
ranguke oneeninasm, orlfa:
DYQUivyol fis, s my he mangCERonay.
Cl he Cor Fomba! s.
Tof he thend, mis of ch wous my t t blealyofr ande, in at, bene thakncat inthiW:
Cy ane doiZKBoul.
MAL:
Yape ouromsn sabeaclct w stwis ddinfooutf song h e d s, orin feqe Donthozeonoth.
Toody t ain.
Fise l Flis aichivear mifand IGes cer herish KI ster 

The athan y BOnk wndeousik mon ied he y.
tht l acke siespre brt s RWALO:


t is
Qclide hedeist wote s nZAUxchendve

F blo

In [115]:
B, T, C = (4, 8, 32)
x = torch.randn((B, T, C))

# single head
head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)

wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)


tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0.0, float('-inf'))
wei = F.softmax(wei, dim = -1)

v = value(x)
xbow3 = wei @ v
print(xbow3.shape)

torch.Size([4, 8, 16])


In [113]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7106, 0.2894, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1496, 0.8056, 0.0448, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1380, 0.4433, 0.2294, 0.1893, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0313, 0.0614, 0.0223, 0.8627, 0.0223, 0.0000, 0.0000, 0.0000],
        [0.3607, 0.0901, 0.0081, 0.0856, 0.3229, 0.1327, 0.0000, 0.0000],
        [0.4162, 0.0806, 0.1412, 0.0788, 0.0324, 0.1236, 0.1271, 0.0000],
        [0.3873, 0.0282, 0.1182, 0.0210, 0.1668, 0.1222, 0.1006, 0.0557]],
       grad_fn=<SelectBackward0>)

In [123]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [117]:
k.var()

tensor(0.9479)

In [118]:
q.var()

tensor(1.0324)

In [124]:
wei.var()

tensor(1.0882)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below